# 1) Import des librairies

In [1]:
from pandas import DataFrame, read_json
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime
import csv
import os

import pandas_profiling

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import random
random.seed(3)

# 2) Contatenation des fichiers json

In [3]:
"""json_files = [pos_json for pos_json in os.listdir() if pos_json.startswith('insta_publi_final')]
insta_total=pd.DataFrame()

for index, js in enumerate(json_files):
    with open(os.path.join(js)) as json_file:
        insta_total = insta_total.append(pd.read_json(js),sort=False)"""

"json_files = [pos_json for pos_json in os.listdir() if pos_json.startswith('insta_publi_final')]\ninsta_total=pd.DataFrame()\n\nfor index, js in enumerate(json_files):\n    with open(os.path.join(js)) as json_file:\n        insta_total = insta_total.append(pd.read_json(js),sort=False)"

In [4]:
"""#POUR SAUVEGARDE AVANT CLEANING POUR METTRE DANS FICHIER IRONHACK
insta_total.to_json('BACKUP/insta_avec_follower_AVANT_CLEANING.json', orient='records')
    #Enregistrer ce fichier de script"""

"#POUR SAUVEGARDE AVANT CLEANING POUR METTRE DANS FICHIER IRONHACK\ninsta_total.to_json('BACKUP/insta_avec_follower_AVANT_CLEANING.json', orient='records')\n    #Enregistrer ce fichier de script"

In [5]:
insta_total = pd.read_json('BACKUP/insta_sans_follower_AVANT_CLEANING.json')

# 3) Premières analyses

la base d'origine contenait des comptes d'institution qui font du livre mais dont ce n'est pas le métier premier(par ex : fondation cartier, le fooding, la cinemathèque de paris...) mais dont le nombre de follower est très grand. Pour que mon apprentissage ne soit pas biaisé par ces éléments, je décide de les supprimer. 

In [6]:
compte_a_supprimer=['fondationcartier','lefooding','fondationlv','phaidonsnaps',
                    'museedartmodernedeparis','openclassroom','cinemathequefr','centreculturelsuisse']
for compte in compte_a_supprimer:
    insta_total.drop(insta_total[insta_total['node.owner.username'] == compte].index, inplace=True)

In [7]:
insta_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322161 entries, 0 to 324176
Data columns (total 32 columns):
page_id                               322161 non-null int64
node.owner.username                   322161 non-null object
date_time                             322161 non-null datetime64[ns]
date                                  322161 non-null datetime64[ns]
code_jour_semaine                     322161 non-null int64
day                                   322161 non-null int64
month                                 322161 non-null int64
year                                  322161 non-null int64
time                                  322161 non-null object
node.id                               322161 non-null int64
node.__typename                       322161 non-null object
text_publication                      322161 non-null object
hashtag                               322161 non-null object
hashtag_count                         322161 non-null int64
node.edge_liked_by.count  

<p style="color:red"> <b>A faire : </b></p> 
commentaire de .info

# 3) Cleaning

## 1) supprimer la colonne `node.edge_media_to_caption.edges` 
Car j'ai déjà récupéré le texte complet dans la colonne `text_publication`

In [8]:
insta_total.drop(columns=['node.edge_media_to_caption.edges'], inplace=True)

## 2) Les Hashtags
- depuis la colonne 'hashtag', je crée une colonne temporaire qui me permet d'isoler pour chaque ligne les hashtags dans une liste de liste de hashtag. Les colonnes sans hashtags sont des listes vides.
- dans un 2ème temps, je renseigne les lignes où il n'y a pas de hashtags par "pas de hashtag". Pour les autres lignes, je crée une liste de hashtags plutot qu'une liste de liste.
- je mets tous les élements de l'étape 2 dans une liste qui alimentera la colonne "hashtag_total" créée
- je supprime les colonnes "hashtag" et "hashtag_temp"

In [9]:
insta_total['hashtag_temp']=insta_total['hashtag'].map(lambda x : [re.findall("#\w+", hashtag) for hashtag in x])

hashtag_total=[]
for liste in insta_total['hashtag_temp']:
    liste_temp=[]
    #test si liste vide alors alors j'écris 'pas de hashtag'
    if len(liste)==0:
        hashtag_total.append('pas de hashtag')
    else :
        for element in liste:
            for words in element:
                liste_temp.append(words)
        hashtag_total.append(liste_temp)

insta_total['hashtag_list'] = hashtag_total

insta_total.drop(columns=['hashtag','hashtag_temp'], inplace=True)

* Je compte maintenant le nombre de hashtag par publication. Le résultat se trouve dans la colonne `hashtag_count`.

In [10]:
insta_total = insta_total.assign(hashtag_list_count=insta_total['hashtag_list'].map(lambda publi: len(publi) if type(publi)==list else 0))
insta_total.drop(columns=['hashtag_count'], inplace=True)

In [11]:
#POUR SAUVEGARDE REGULIERE
insta_total.to_json('BACKUP/insta_publi_final-BACKUP.json', orient='records')
    #Enregistrer ce fichier de script

## 3) Les publications (colonne : `text_publication`)
Afin de pouvoir analyser au mieux le texte publié, je crée une colonne (`publication_word_list`) avec le texte des publications mais sans les hashtags. <br/> J'en profite pour créer également un colonne (`publication_word_list_count`) pour compter le nombre de mots restants.


In [12]:
insta_total['publication_word_list']=insta_total['text_publication'].map(lambda x : re.sub(r"#\w+", r"", x).split() if x!='Pas de texte' else '')

insta_total = insta_total.assign(publication_word_list_count=insta_total['publication_word_list'].map(lambda publi: len(publi) if type(publi)==list else 0))


## 4) Je créé une colonne `hour` à partir de la colonne `time`

In [13]:
insta_total['hour'] = pd.to_datetime(insta_total['time']).apply(lambda x: x.hour)

In [14]:
insta_total.columns

Index(['page_id', 'node.owner.username', 'date_time', 'date',
       'code_jour_semaine', 'day', 'month', 'year', 'time', 'node.id',
       'node.__typename', 'text_publication', 'node.edge_liked_by.count',
       'node.dimensions.height', 'node.dimensions.width', 'node.display_url',
       'node.edge_media_preview_like.count',
       'node.edge_media_to_comment.count', 'node.video_view_count',
       'node.fact_check_information', 'node.gating_info', 'node.is_video',
       'node.location', 'node.location.has_public_page', 'node.location.id',
       'node.location.name', 'node.location.slug', 'node.shortcode',
       'node.thumbnail_src', 'hashtag_list', 'hashtag_list_count',
       'publication_word_list', 'publication_word_list_count', 'hour'],
      dtype='object')

## 5) Suppression des duplicates.
Avec un rythme de publications qui diffèrent d'un compte à l'autre, le recueil de la données sur un mois a apporté l'éventuealité d'avoir des doublons. Je vais donc supprimer les élements dupliquer et ne garder que la première occurence c'est à dire la ligne qui a été recueillie la première fois.
Chaque publication ayant un ID (clé unique) identfié dans la colonne `node.id`, ce dernier sera l'élément par lequel je supprime les duplicates.

In [15]:
insta_total.shape

(322161, 34)

In [16]:
insta_total.drop_duplicates(subset='node.id', keep='first', inplace=True)
insta_total.shape

(23273, 34)

Je passe de 324k lignes à 23k lignes pour la suite du travail.

## 6) Création d'une colonne Semaine (0) /weekend (1)

In [17]:
insta_total = insta_total.assign(week_weekend=insta_total['code_jour_semaine'].map(lambda day: 1 if day in [6,7] else 0))

## 7) Je renomme les colonnes qui commencent par node pour une meilleure lecture de la dataframe

In [18]:
insta_total.rename(columns={
    'node.owner.username':'page_username',
    'node.id': 'publication_id',
    'node.__typename': 'publication_type',
    'node.edge_liked_by.count':'publication_liked_count',
    'node.dimensions.height':'publication_height',
    'node.dimensions.width':'publication_width',
    'node.display_url':'publication_url',
    'node.edge_media_preview_like.count':'publication_preview_liked_count',  
    'node.edge_media_to_comment.count':'publication_comment_count',
    'node.video_view_count':'publication_video_view_count',
    'node.fact_check_information':'publication_fact_check_information',
    'node.gating_info':'gating_info',
    'node.is_video':'publication_is_video',
    'node.location':'publication_location',
    'node.location.has_public_page':'publication_location_has_public_page',
    'node.location.id':'publication_location_id',
    'node.location.name':'publication_location_name',
    'node.location.slug':'publication_location_slug',
    'node.shortcode':'publication_shortcode',
    'node.thumbnail_src':'publication_thumbnail_src'}, inplace=True)

## 8) Ajout d'une colonne nb de followers de la page

In [19]:
"""json_files = [pos_json for pos_json in os.listdir() if pos_json.startswith('insta_total') and pos_json.endswith('json')]
insta_fo=pd.DataFrame()

for index, js in enumerate(json_files):
    with open(os.path.join(js)) as json_file:
        insta_fo = insta_fo.append(pd.read_json(js),sort=False)"""

insta_fo = pd.read_json('BACKUP/insta_ajout_followers_POUR_CLEANING.json')

insta_fo.drop_duplicates(subset='node.id', keep='first', inplace=True)
insta_fo.rename(columns={'node.id':'publication_id'}, inplace=True)

insta_total = pd.merge(insta_total, insta_fo[['graphql.user.edge_followed_by.count','publication_id']], on='publication_id', how='left')

insta_total.rename(columns={'graphql.user.edge_followed_by.count':'page_nb_followers'}, inplace=True)


## 9) Ajout d'une colonne edi_lib
Cette colonne sert à distinguer s'il s'agit d'un compte d'un éditeur (0) ou d'un libraire (1)

In [20]:
edi_lib = pd.read_csv('BDD_EDITEURS_LIBRAIRIES-FIN_AOUT_2019.csv')
editeur=[page for acteur,page in zip(edi_lib['ACTEUR'],edi_lib['url_rs_ID']) if acteur=='EDITEUR' or acteur=='edITEUR']
libraire = [page for acteur,page in zip(edi_lib['ACTEUR'],edi_lib['url_rs_ID']) if acteur=='LIBRAIRIE']
editeur=set(editeur)
libraire=set(libraire)

insta_total = insta_total.assign(edi_lib=insta_total['page_username'].map(lambda x: 0 if x in editeur else 1 if x in libraire else ''))

list_edi = set([y for y in insta_total.loc[insta_total['edi_lib']=='']['page_username']])

for y in list_edi:
    insta_total['edi_lib'].loc[insta_total['page_username']==y]=0

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## 10) Je transforme les données **NaN** de la colonne `publication_video_view_count` en 0

In [21]:
insta_total['publication_video_view_count'].loc[insta_total['publication_video_view_count'].isnull()]=0

In [22]:
insta_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23273 entries, 0 to 23272
Data columns (total 37 columns):
page_id                                 23273 non-null int64
page_username                           23273 non-null object
date_time                               23273 non-null datetime64[ns]
date                                    23273 non-null datetime64[ns]
code_jour_semaine                       23273 non-null int64
day                                     23273 non-null int64
month                                   23273 non-null int64
year                                    23273 non-null int64
time                                    23273 non-null object
publication_id                          23273 non-null int64
publication_type                        23273 non-null object
text_publication                        23273 non-null object
publication_liked_count                 23273 non-null int64
publication_height                      23273 non-null int64
publication_wid

## 11) Suppression de colonnes vides
J'ai 3 colonnes qui sont vides : publication_fact_check_information, gating_info, publication_location --> je vais les supprimer, il me restera donc 32 colonnes
Je fais le choix de supprimer les colonnes relatives à la localisation car je n'ai que 28% de lignes renseignées pour chacune de ces colonnes.

In [23]:
insta_total.drop(columns=['publication_fact_check_information','gating_info',
                          'publication_location','publication_location_has_public_page','publication_location_id',
                          'publication_location_name','publication_location_slug'], inplace=True)

In [26]:
#POUR SAUVEGARDE REGULIERE
insta_total.to_json('BACKUP/insta_avec_follower_final-BACKUP.json', orient='records')
    #Enregistrer ce fichier de script

In [25]:
"""%%time
pandas_profiling.ProfileReport(insta_total)"""

'%%time\npandas_profiling.ProfileReport(insta_total)'